# Amazon Home and Kitchen Products Review-Sentiment Analysis-Data Wrangling

### Import Necessary Libraries

In [1]:
# Dataframe
import pandas as pd

# Array
import numpy as np

# Decompress the file
import gzip

# Visualizations
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import seaborn as sns
import matplotlib.colors as colors
%matplotlib inline

# Datetime
from datetime import datetime

## Warnings
import warnings
from scipy import stats
warnings.filterwarnings('ignore')

### Decompress the Amazon Review Dataset
The dataset is obtained from http://seotest.ciberius.info/seo--snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Home_and_Kitchen_10.json.gz link and uploaded into Jupyter notebook via the following code.

In [2]:
def parse(path):
  g = gzip.open(path, 'rb')
  for l in g:
    yield eval(l)

def getDF(path):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
  return pd.DataFrame.from_dict(df, orient='index')

df = getDF('D:\GENEL\SPRINGBOARD\PROJECTS\CAPSTONE PROJECTS\CAPSTONE PROJECT-2/reviews_Home_and_Kitchen_10.json.gz')

### Inspecting the Dataset

In [3]:
# Shape of the dataset
df.shape

(25445, 9)

In [4]:
# Read the file
df.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A1115ST6F5CWYP,B00000JGRT,Amalfi Coast Girl,"[29, 33]",I have had one of these for about 10 years. I...,4.0,good for a first ice cream machine,1148256000,"05 22, 2006"
1,A188JOXWF4EY1R,B00000JGRT,"Ann B. Hibbard ""anbee""","[4, 4]",We actually found this product on clearance sa...,4.0,Wonderful Product!,1282176000,"08 19, 2010"
2,AUAX1QWUCYKSX,B00000JGRT,Ashley S,"[1, 1]","This product works great, if the unit kept in ...",5.0,Works as expected,1243555200,"05 29, 2009"
3,A2C27IQUH9N1Z,B00000JGRT,audrey,"[12, 13]",After trying other ice cream makers with mixed...,5.0,this will be one of your favorite small applia...,1043712000,"01 28, 2003"
4,A2PN65B6BSTIYZ,B00000JGRT,B. A. Chaney,"[1, 1]",I bought this ice cream maker last summer and ...,5.0,You'll be addicted to homemade ice cream!,1214179200,"06 23, 2008"


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25445 entries, 0 to 25444
Data columns (total 9 columns):
reviewerID        25445 non-null object
asin              25445 non-null object
reviewerName      25276 non-null object
helpful           25445 non-null object
reviewText        25445 non-null object
overall           25445 non-null float64
summary           25445 non-null object
unixReviewTime    25445 non-null int64
reviewTime        25445 non-null object
dtypes: float64(1), int64(1), object(7)
memory usage: 1.9+ MB


#### Each row corresponds to a customers' review information and related variables are explained below:

**reviewerID :** ID of the reviewer

**asin :** ID of the product

**reviewerName :** name of the reviewer

**helpful :** helpfulness of the review, e.g. 2/3

**reviewText :** text of the review

**overall :** rating

**summary :** summary of the review

**unixReviewTime :** time of the review (unix time)

**reviewTime :** time of the review (raw)

In [6]:
# Inspect a sample 'reviewText'
df['reviewText'][1001]

'I bought this Food Chopper to replace an old Cuisinart that was starting to come apart at the blade.  So far, I am not impressed.  I attempted to make hummus and it kept freezing up.  I guess the beans were just too thick.  I tried to thin the beans out with olive oil, remove some, etc., and it still froze up and I did not have any more than the 3 cup limit in the food chopper.For the money, I should have stayed with another Cuisinart.  My Cuisinart held up for about 8 years before the blade stem starting splitting.  I may still order a Cuisinart and keep the Black & Decker for more liquid dishes.'

In [7]:
# Inspect a sample 'summary'
df['summary'][1001]

'I think I should have stuck with my Cuisinart'

In [8]:
# Inspect a sample 'overall'
df['overall'][1001]

3.0

In [9]:
# Basic Statistics of the Dataset
df.describe()

,overall,unixReviewTime
count,25445.000000,2.544500e+04
mean,4.292867,1.356088e+09
std,0.914940,5.240118e+07
min,1.000000,9.588672e+08
25%,4.000000,1.340842e+09
50%,5.000000,1.367453e+09
75%,5.000000,1.392854e+09
max,5.000000,1.406074e+09


In [10]:
# Customer totals for each rating 
df['overall'].value_counts()

5.0    13342
4.0     7919
3.0     2897
2.0      868
1.0      419
Name: overall, dtype: int64

In [11]:
# Check for missing values
df.isnull().sum()

reviewerID          0
asin                0
reviewerName      169
helpful             0
reviewText          0
overall             0
summary             0
unixReviewTime      0
reviewTime          0
dtype: int64

### What we learned so far:

Some information about dataset so far:

This data includes 25445 rows(observations) and 9 columns(feature variables).

Memory usage is 1.9+ MB.

We have 7 object, 1 float64 and 1 int64 data types.

169 'reviewerName' information is missing in the dataset. Since customer don't give their name, we should be suspicious about their reviews and ratings. I would prefer to drop the missing values from dataset since we have enough observations to conclude a decision for sentiment analysis.

We should concatenate 'reviewText' and 'summary' since both of them gives information about product, and also drop both 'reviewText' and 'summary' column at the end.

'helpful' variable includes positive and negative feedback for reviews' information, it will be splitted into two columns: positive feedback will be represented as "pos_feedback" , and negative feedback will be represented as "neg_feedback". Also, 'helpful' variable will be represented as an int64 type in the dataset via that step.

We should classify the 'overall' (ratings) as good and bad in the new "rating_class" column in order to make sentiment analysis.

In the dataset, 'reviewerID' and 'reviwerName' are both for identification of customer. We should drop one of them from the dataset. Preferably, I will drop 'reviewerName' since customer names are not standardized and there are lots of different sytle to represent them.

'unixReviewTime' will be dropped since it has already been represented in 'reviewTime' feature in a more understandable format. Also, 'reviewTime' will be converted to datetime data type.

We will rename the columns in order to improve practicality and the readibility of coding as stated below:

**reviewerID :** "customer"

**asin :** "product"

**reviewerName :** column will be dropped.

**helpful :** positive feedback will be represented as "pos_feedback" and negative feedback will be represented as "neg_feedback".

**reviewText :** This will be concatenated with "summary" and renamed as "review_text"

**overall :** "rating"

**summary :** it will be dropped after it is concatenated with "reviewerText".

**unixReviewTime :** column will be dropped.

**reviewTime :** "time"

Now, we will practice all defined action items above.

### Dropping Missing Values

In [12]:
# Drop missing customers who don't give their names
df2 = df.dropna(axis=0)

In [13]:
df2.isnull().sum()

reviewerID        0
asin              0
reviewerName      0
helpful           0
reviewText        0
overall           0
summary           0
unixReviewTime    0
reviewTime        0
dtype: int64

In [14]:
df2.shape

(25276, 9)

In [15]:
# Customer totals for each rating (after drop missing values)
df2['overall'].value_counts()

5.0    13264
4.0     7869
3.0     2867
2.0      861
1.0      415
Name: overall, dtype: int64

### Concatenating 'reviewText' and 'summary' columns

In [16]:
# Concatenate 'reviewText' and 'summary' as review_text and drop them 
df2['review_text'] = df[['summary', 'reviewText']].apply(lambda x: " ".join(str(y) for y in x if str(y) != 'nan'), axis = 1)
df2 = df2.drop(['reviewText', 'summary'], axis = 1)
df2.head()

,reviewerID,asin,reviewerName,helpful,overall,unixReviewTime,reviewTime,review_text
0,A1115ST6F5CWYP,B00000JGRT,Amalfi Coast Girl,"[29, 33]",4.0,1148256000,"05 22, 2006",good for a first ice cream machine I have had ...
1,A188JOXWF4EY1R,B00000JGRT,"Ann B. Hibbard ""anbee""","[4, 4]",4.0,1282176000,"08 19, 2010",Wonderful Product! We actually found this prod...
2,AUAX1QWUCYKSX,B00000JGRT,Ashley S,"[1, 1]",5.0,1243555200,"05 29, 2009","Works as expected This product works great, if..."
3,A2C27IQUH9N1Z,B00000JGRT,audrey,"[12, 13]",5.0,1043712000,"01 28, 2003",this will be one of your favorite small applia...
4,A2PN65B6BSTIYZ,B00000JGRT,B. A. Chaney,"[1, 1]",5.0,1214179200,"06 23, 2008",You'll be addicted to homemade ice cream! I bo...


In [17]:
df2['review_text'][1001]

'I think I should have stuck with my Cuisinart I bought this Food Chopper to replace an old Cuisinart that was starting to come apart at the blade.  So far, I am not impressed.  I attempted to make hummus and it kept freezing up.  I guess the beans were just too thick.  I tried to thin the beans out with olive oil, remove some, etc., and it still froze up and I did not have any more than the 3 cup limit in the food chopper.For the money, I should have stayed with another Cuisinart.  My Cuisinart held up for about 8 years before the blade stem starting splitting.  I may still order a Cuisinart and keep the Black & Decker for more liquid dishes.'

### Spliting 'helpful' column into two separate columns

In [18]:
# Split 'helpful' column into 'neg_feedback' and 'pos_feedback' and drop it at the end

list_positive = []
list_negative = []

for help in df2['helpful']:
    list_positive.append(help[0])
    list_negative.append(help[1] - help[0])
    
# Positive Feedback (Supporting reviewer's idea and find it useful)
df2['pos_feedback'] = list_positive

# Negative Feedback (Finding reviewer's idea useless and not supporting it)
df2['neg_feedback'] = list_negative

# Drop redundant 'helpful' column
df2 = df2.drop('helpful', axis = 1)
df2.head()

,reviewerID,asin,reviewerName,overall,unixReviewTime,reviewTime,review_text,pos_feedback,neg_feedback
0,A1115ST6F5CWYP,B00000JGRT,Amalfi Coast Girl,4.0,1148256000,"05 22, 2006",good for a first ice cream machine I have had ...,29,4
1,A188JOXWF4EY1R,B00000JGRT,"Ann B. Hibbard ""anbee""",4.0,1282176000,"08 19, 2010",Wonderful Product! We actually found this prod...,4,0
2,AUAX1QWUCYKSX,B00000JGRT,Ashley S,5.0,1243555200,"05 29, 2009","Works as expected This product works great, if...",1,0
3,A2C27IQUH9N1Z,B00000JGRT,audrey,5.0,1043712000,"01 28, 2003",this will be one of your favorite small applia...,12,1
4,A2PN65B6BSTIYZ,B00000JGRT,B. A. Chaney,5.0,1214179200,"06 23, 2008",You'll be addicted to homemade ice cream! I bo...,1,0


### Classify the 'overall' (ratings) as good and bad

In [19]:
# Classify ratings as good
good_rate = len(df2[df2['overall'] >= 3])
bad_rate = len(df2[df2['overall'] < 3])

# Printing rates and their total numbers
print ('Good ratings : {} reviews for beauty products'.format(good_rate))
print ('Bad ratings : {} reviews for beauty products'.format(bad_rate))

Good ratings : 24000 reviews for beauty products
Bad ratings : 1276 reviews for beauty products


In [20]:
# Apply the new classification to the ratings column
df2['rating_class'] = df2['overall'].apply(lambda x: 'bad' if x < 3 else'good')
df2.head()

,reviewerID,asin,reviewerName,overall,unixReviewTime,reviewTime,review_text,pos_feedback,neg_feedback,rating_class
0,A1115ST6F5CWYP,B00000JGRT,Amalfi Coast Girl,4.0,1148256000,"05 22, 2006",good for a first ice cream machine I have had ...,29,4,good
1,A188JOXWF4EY1R,B00000JGRT,"Ann B. Hibbard ""anbee""",4.0,1282176000,"08 19, 2010",Wonderful Product! We actually found this prod...,4,0,good
2,AUAX1QWUCYKSX,B00000JGRT,Ashley S,5.0,1243555200,"05 29, 2009","Works as expected This product works great, if...",1,0,good
3,A2C27IQUH9N1Z,B00000JGRT,audrey,5.0,1043712000,"01 28, 2003",this will be one of your favorite small applia...,12,1,good
4,A2PN65B6BSTIYZ,B00000JGRT,B. A. Chaney,5.0,1214179200,"06 23, 2008",You'll be addicted to homemade ice cream! I bo...,1,0,good


### Dropping duplicate columns about customer ('reviwerName') and time ('unixReviewTime)

In [21]:
# Drop 'reviewName' and 'unixReviewTime'columns
df2 = df2.drop(['reviewerName', 'unixReviewTime'], axis = 1)
df2.head()

,reviewerID,asin,overall,reviewTime,review_text,pos_feedback,neg_feedback,rating_class
0,A1115ST6F5CWYP,B00000JGRT,4.0,"05 22, 2006",good for a first ice cream machine I have had ...,29,4,good
1,A188JOXWF4EY1R,B00000JGRT,4.0,"08 19, 2010",Wonderful Product! We actually found this prod...,4,0,good
2,AUAX1QWUCYKSX,B00000JGRT,5.0,"05 29, 2009","Works as expected This product works great, if...",1,0,good
3,A2C27IQUH9N1Z,B00000JGRT,5.0,"01 28, 2003",this will be one of your favorite small applia...,12,1,good
4,A2PN65B6BSTIYZ,B00000JGRT,5.0,"06 23, 2008",You'll be addicted to homemade ice cream! I bo...,1,0,good


### Arranging Time column

In [22]:
# Convert time object to datetime and create a new column named 'time'
df2['time'] = df2.reviewTime.str.replace(',', "")
df2['time'] = pd.to_datetime(df2['time'], format = '%m %d %Y')

# Drop redundant 'reviewTime' column
df2 = df2.drop('reviewTime', axis = 1)
df2.head()

,reviewerID,asin,overall,review_text,pos_feedback,neg_feedback,rating_class,time
0,A1115ST6F5CWYP,B00000JGRT,4.0,good for a first ice cream machine I have had ...,29,4,good,2006-05-22
1,A188JOXWF4EY1R,B00000JGRT,4.0,Wonderful Product! We actually found this prod...,4,0,good,2010-08-19
2,AUAX1QWUCYKSX,B00000JGRT,5.0,"Works as expected This product works great, if...",1,0,good,2009-05-29
3,A2C27IQUH9N1Z,B00000JGRT,5.0,this will be one of your favorite small applia...,12,1,good,2003-01-28
4,A2PN65B6BSTIYZ,B00000JGRT,5.0,You'll be addicted to homemade ice cream! I bo...,1,0,good,2008-06-23


### Rename column names

In [23]:
# Rename the columns
df2.columns = ['customer', 'product', 'rating', 'review_text', 'pos_feedback', 'neg_feedback', 'rating_class', 'time']

# Sample observation
df2.head(5)

,customer,product,rating,review_text,pos_feedback,neg_feedback,rating_class,time
0,A1115ST6F5CWYP,B00000JGRT,4.0,good for a first ice cream machine I have had ...,29,4,good,2006-05-22
1,A188JOXWF4EY1R,B00000JGRT,4.0,Wonderful Product! We actually found this prod...,4,0,good,2010-08-19
2,AUAX1QWUCYKSX,B00000JGRT,5.0,"Works as expected This product works great, if...",1,0,good,2009-05-29
3,A2C27IQUH9N1Z,B00000JGRT,5.0,this will be one of your favorite small applia...,12,1,good,2003-01-28
4,A2PN65B6BSTIYZ,B00000JGRT,5.0,You'll be addicted to homemade ice cream! I bo...,1,0,good,2008-06-23


### Descriptive Statistics

In [24]:
# Read statistic summary of numeric variables
df2.describe()

,rating,pos_feedback,neg_feedback
count,25276.000000,25276.000000,25276.000000
mean,4.293955,5.253956,0.704977
std,0.914244,63.352885,3.006284
min,1.000000,0.000000,0.000000
25%,4.000000,0.000000,0.000000
50%,5.000000,1.000000,0.000000
75%,5.000000,2.000000,1.000000
max,5.000000,6128.000000,207.000000


In [25]:
# Customer totals for each rating 
df2['rating'].value_counts()

5.0    13264
4.0     7869
3.0     2867
2.0      861
1.0      415
Name: rating, dtype: int64

In [26]:
# Customer totals for each rating class
df2['rating_class'].value_counts()

good    24000
bad      1276
Name: rating_class, dtype: int64

In [27]:
# Print the number of positive feedback
pos = len(df2[df2['pos_feedback'] > 0])
print('Number of positive feedbacks: {}'.format(pos))

# Print the number of negative feedback
neg = len(df2[df2['neg_feedback'] > 0])      
print('\nNumber of negative feedbacks: {}'.format(neg))

Number of positive feedbacks: 13298

Number of negative feedbacks: 8147


In [28]:
df3 = df2[df2['pos_feedback'] == 0]
df4 = df3[df3['neg_feedback']==0]
df4.shape

(10230, 8)

In [29]:
# Statistics of non-numeric variables

# Number of unique customers
print('\nNumber of unique customers : {}'.format(len(df2['customer'].unique())))
      
# Number of unique products
print('\nNumber of unique products : {}'.format(len(df2['product'].unique())))
      
# Review number per unique customer
print('\nReview per customer: {}'.format((len(df2)/len(df2['customer'].unique()))))      

# Review number per unique product 
print('\nReview per product: {}'.format((len(df2)/len(df2['product'].unique()))))


Number of unique customers : 1395

Number of unique products : 1171

Review per customer: 18.11899641577061

Review per product: 21.584970111016226


### Descriptive Statistic Summary:

** - Rating Status:**

28576 customer gives ratings and mean of the ratings is 4.17, which means that customers prefer to give high ratings for products. Standard deviation and percentiles also show that 1 and 2 ratings for products are rare. To be able to predict the ratings reasonably, we classified them as 'good', 'neutral', and 'bad' above.

According to the statistics on rating stars:

415 customers give 1 star

861 customers give 2 stars

2867 customers give 3 stars

7869 customers give 4 stars

13264 customers give 5 stars

On the other hand, if we look for the rating class, the picture is look like below.

1276 customers give bad ratings

24000 customers give good ratings

**- Feedback Status**

13298 customers find the given reviews logical and totally agree with them. They give positive feedbacks. The mean 5.25 and standard deviation is 63.35 whereas the highest positive feedback number for reviews is 6128. 

8147 customers find the given reviews useless and don't aggree with the them. They give negative feedbacks. The mean 0.7 and standard deviation is 3.0 whereas the highest negative feedback number for reviews is 207.

On the other hand 3831 customers don't give either positive or negative feedbacks for reviews. 

10230 customers don't give either positive or negative feedbacks for reviews.

**- Non-numeric variables statistics:**

We have 1395 unique customers and 1171 products in this dataset. Each customers averagely give 18 reviews for products and on the other hand, there is averagely 22 reviews for each product in the website.

On exploratory data analysis, we will check if there is a case for outliers by visualizing data. 

### Text Preprocessing

In [30]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords
from bs4 import BeautifulSoup
import unicodedata
import contractions
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize, sent_tokenize, regexp_tokenize 
from nltk.stem import PorterStemmer, WordNetLemmatizer
import re

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [32]:
# Let's put aside number of raw tokens in order to measure of cleaned tokens
from nltk.tokenize import word_tokenize
raw_tokens=len([w for t in (df2["review_text"].apply(word_tokenize)) for w in t])
print('Number of raw tokens: {}'.format(raw_tokens))

Number of raw tokens: 6623123


### Functions for Preprocessing

In [33]:
# html_tags removal
def strip_html_tags(text):
    soup = BeautifulSoup(text, "html.parser")
    stripped_text = soup.get_text()
    return stripped_text

# accented_chars removal
def remove_accented_chars(text):
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    return text

# special_characters removal
def remove_special_characters(text, remove_digits=False):
    pattern = r'[^a-zA-z0-9\s]' if not remove_digits else r'[^a-zA-z\s]'
    text = re.sub(pattern, '', text)
    return text


# Lemmatization
lemmatizer=WordNetLemmatizer()
def lemmatize_text(text):
    text = " ".join(lemmatizer.lemmatize(w) for w in text.split(' '))
    return text

# Creating stopwordlist and editing 
stopword_list= stopwords.words('english')
    
# "no" and "not" may give us information so those are removed from stop lists
stopword_list.remove('no')
stopword_list.remove('not')
                     
def remove_stopwords(text, stop_words=stopword_list):
    tokens = [w for w in nltk.word_tokenize(text) if w not in stop_words]
    return ' '.join(tokens)    

                     

def normalize_text(text, html_stripping=True, contraction_expansion=True,
                     accented_char_removal=True, text_lower_case=True, 
                     text_lemmatization=True, special_char_removal=True, 
                     stopword_removal=True, remove_digits=True):
    
    # strip HTML
    if html_stripping:
        text = strip_html_tags(text)
        
    # expand contractions    
    if contraction_expansion:
        text = contractions.fix(text)
        
        # remove accented characters
    if accented_char_removal:
        text = remove_accented_chars(text)

    # lowercase the text    
    if text_lower_case:
        text = text.lower()
    
    # remove extra newlines
    text = re.sub(r'[\r|\n|\r\n]+', ' ',text)
    
    # remove special characters and\or digits    
    if special_char_removal:
        # insert spaces between special characters to isolate them    
        special_char_pattern = re.compile(r'([{.(-)!}])')
        text = special_char_pattern.sub(" \\1 ", text)
        text = remove_special_characters(text, remove_digits=remove_digits)  
    
    # remove extra whitespace and underscore
    text = re.sub(' +', ' ', text)
    text = re.sub('_+', '', text)
    
    # lemmatize text
    if text_lemmatization:
        text = lemmatize_text(text)
        
    # remove stopwords
    if stopword_removal:
        text = remove_stopwords(text)
       
    return text

### Cleaning the Text

In [34]:
df2['clean_text'] = df2['review_text'].map(lambda text: normalize_text(text))

In [35]:
# Let's put aside number of raw tokens in order to measure of cleaned tokens
from nltk.tokenize import word_tokenize
clean_tokens=len([w for t in (df2["clean_text"].apply(word_tokenize)) for w in t])
print('Number of clean tokens: {}\n'.format(clean_tokens))
print('Percentage of removed tokens: {0:.2f}'.format(1-(clean_tokens/raw_tokens)))

Number of clean tokens: 3070479

Percentage of removed tokens: 0.54


### Saving the cleaned data

In [36]:
df2.to_csv('D:\GENEL\SPRINGBOARD\PROJECTS\CAPSTONE PROJECTS\CAPSTONE PROJECT-2/Cleaned_Reviews_Home_and_Kitchen.csv', sep=',', encoding='utf-8', index = False)